In [3]:
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns
from pandas_profiling import ProfileReport

In [4]:
subscribers = pd.read_csv('subscribers_update.csv')

customers = subscribers[['subid','package_type',
                         'preferred_genre', 'intended_use', 'weekly_consumption_hour',
                          'age', 'male_TF', 'plan_type',
                         'monthly_price', 'join_fee', 'current_sub_TF', 'attribution_technical',
                         'attribution_survey', 
                         'payment_period', 'trial_completed']]


customers.dropna(inplace=True)
customers = customers.drop(customers[customers['age']>100].index)
customers = customers.drop(customers[customers['weekly_consumption_hour']<0].index)
customers = customers.drop(customers[customers['join_fee']<0].index)
customers = customers.drop(customers[customers['age']<5].index)
#customers.drop(customers.index[customers['age'] > 100], inplace = True)

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [5]:
subscribers.head()

,Unnamed: 0,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,...,join_fee,language,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed
0,162502,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,...,NaN,ar,Standard Charter,14,True,1,2020-02-07 21:44:16,2020-06-07 21:44:16,NaN,True
1,380374,23383224,base,NaN,comedy,access to exclusive content,22.951579,NaN,70.0,False,...,0.3303,ar,NaN,14,True,1,2020-03-15 15:44:35,2020-07-15 15:44:35,NaN,True
2,496617,26844789,enhanced,3.0,regional,replace OTT,36.001186,2.0,25.0,True,...,0.1101,ar,NaN,14,False,0,NaN,NaN,NaN,True
3,356799,29417030,base,NaN,drama,replace OTT,20.051667,NaN,30.0,False,...,0.0367,ar,NaN,14,False,0,NaN,NaN,NaN,True
4,681701,26723159,base,4.0,comedy,replace OTT,22.951579,3.0,28.0,False,...,0.0367,ar,NaN,14,True,2,2020-02-19 12:57:07,2020-06-19 12:57:07,NaN,True


In [ ]:


customers_dummy = pd.get_dummies(data=customers, columns=['package_type', 'preferred_genre',
                                                    'intended_use', 'male_TF', 'plan_type',
                                                    'current_sub_TF', 'trial_completed', 
                                                    'attribution_technical','attribution_survey',])



In [ ]:
x = customers_dummy.values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

inertias = {}
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, random_state=2020)
    kmeans.fit(df)
    inertias[k] = kmeans.inertia_
print(inertias)


ax = plt.subplot()
ax.plot(list(inertias.keys()), list(inertias.values()), '-*')
ax.set_xticks(np.arange(1, 20))
ax.grid()
plt.show()

In [ ]:
customers.

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=2020)
y_pred = kmeans.fit_predict(df)
y_pred = list(y_pred)

customers['cluster'] = y_pred

customers.to_csv('customers_cluster result.csv')

reduced_data = PCA(n_components=2).fit_transform(df)
results = pd.DataFrame(reduced_data, columns=['pca1', 'pca2'])

sns.scatterplot(x="pca1", y="pca2", hue=y_pred, data=results)
plt.title('K-means Clustering with 3 dimensions')
plt.show()


In [ ]:
n_components = 3
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X_scaled)

In [ ]:
pd.DataFrame.from_dict(inertias , orient='index')

In [ ]:
customers

In [ ]:
customers.to_csv('customer_cluster_sys.csv', index=False)

### Churn Modeling

In [ ]:
import pandas as pd
subscribers = pd.read_csv('subscribers_update.csv')

In [ ]:
subscribers['churn'] = ''

In [ ]:
subscribers.drop('Unnamed: 0',axis=1,inplace=True)
pd.set_option('display.max_columns',None)
subscribers.head(10)

#### Define Churn
    1.No payment & (not current subscriber | no cancel date) is churn
    2.Have payment is not churn
    3.others are all not churn

In [7]:

subchurn = pd.read_csv('subscribers_ churn.csv')

In [8]:
subchurn.head()

,subid,package_type,num_weekly_services_utilized,preferred_genre,intended_use,weekly_consumption_hour,num_ideal_streaming_services,age,male_TF,country,...,payment_type,num_trial_days,current_sub_TF,payment_period,last_payment,next_payment,cancel_date,trial_completed,change,churn
0,21724479,economy,NaN,comedy,access to exclusive content,NaN,NaN,NaN,False,UAE,...,Standard Charter,14,True,1,2020/2/7 21:44,2020/6/7 21:44,NaN,True,NaN,0
1,20228345,enhanced,3.0,regional,replace OTT,33.101274,2.0,100.0,False,UAE,...,NaN,14,False,0,NaN,NaN,2019/10/30 0:00,False,-1.0,1
2,25045794,enhanced,NaN,drama,access to exclusive content,21.501623,NaN,99.0,False,UAE,...,NaN,14,False,0,NaN,NaN,2020/2/19 0:00,False,0.0,1
3,28496228,enhanced,NaN,comedy,access to exclusive content,24.401536,NaN,96.0,False,UAE,...,NaN,14,True,0,NaN,2020/4/9 19:38,2020/3/29 0:00,False,0.0,1
4,23301935,economy,NaN,comedy,replace OTT,22.951579,NaN,92.0,False,UAE,...,NaN,14,False,0,NaN,NaN,NaN,True,0.0,1


In [9]:
cuschurn = subchurn[['package_type',
                         'preferred_genre', 'intended_use', 'weekly_consumption_hour',
                          'age', 'male_TF', 'plan_type',
                         'monthly_price', 'join_fee', 'attribution_technical',
                         'attribution_survey', 'op_sys', 'discount_price'
                         , 'churn']]


cuschurn.dropna(inplace=True)
cuschurn = cuschurn.drop(cuschurn[cuschurn['age']>100].index)
cuschurn = cuschurn.drop(cuschurn[cuschurn['weekly_consumption_hour']<0].index)
cuschurn = cuschurn.drop(cuschurn[cuschurn['join_fee']<0].index)
cuschurn = cuschurn.drop(cuschurn[cuschurn['age']<5].index)

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
cuschurn['churn'].value_counts()

0    91257
1    46358
Name: churn, dtype: int64

In [11]:
cuschurn_dummy = pd.get_dummies(data=cuschurn, columns=['package_type','preferred_genre', 
                                                      'intended_use',
                                                       'male_TF', 'attribution_technical',
                                                      'attribution_survey',
                                                      'op_sys','plan_type'])

In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cuschurn_dummy, test_size=0.2)
train, vali = train_test_split(train, test_size=0.3)

In [13]:
print(train.shape)
print(vali.shape)
print(test.shape)

(77064, 79)
(33028, 79)
(27523, 79)


In [14]:
cuschurn_dummy

,weekly_consumption_hour,age,monthly_price,join_fee,discount_price,churn,package_type_base,package_type_economy,package_type_enhanced,preferred_genre_comedy,...,attribution_survey_tv,attribution_survey_youtube,op_sys_Android,op_sys_iOS,plan_type_base_eur_14_day_trial,plan_type_base_uae_14_day_trial,plan_type_high_aud_14_day_trial,plan_type_high_sar_14_day_trial,plan_type_high_uae_14_day_trial,plan_type_low_gbp_14_day_trial
1,33.101274,100.0,4.7343,0.0367,4.5141,1,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
2,21.501623,99.0,4.7343,0.0367,4.5141,1,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
3,24.401536,96.0,4.7343,0.0000,4.5141,1,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0
6,25.851492,91.0,4.7343,0.0000,4.5141,1,1,0,0,1,...,0,0,1,0,0,1,0,0,0,0
10,22.951579,89.0,4.7343,0.0367,4.5141,1,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219661,24.401536,18.0,4.7343,0.0367,4.5141,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
219664,22.951579,18.0,4.7343,0.3303,4.5141,0,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0
219665,25.851492,18.0,4.7343,0.0367,4.5141,0,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0
219668,24.401536,18.0,4.7343,0.0367,4.5141,0,0,0,1,1,...,0,0,0,1,0,1,0,0,0,0


In [28]:
import xgboost as xgb
from sklearn import metrics 

In [24]:
train_y

51962     1
27030     1
199845    0
138673    0
37852     1
         ..
20829     0
176996    0
58004     1
213740    0
62088     0
Name: intended_use_access to exclusive content, Length: 77064, dtype: uint8

In [16]:
train_x = train.drop(['churn'], axis=1)
train_y = train.iloc[:,5]

vali_x = vali.drop(['churn'], axis=1)
vali_y = vali.iloc[:,5]

test_x = test.drop(['churn'], axis=1)
test_y = test.iloc[:,5]

In [25]:
x_churn = cuschurn_dummy.drop(['churn'], axis=1)
churn_prob = clf.predict_proba(x_churn)[:,1]
churn_prob

array([9.87680436e-01, 9.24224123e-01, 5.28328591e-01, ...,
       3.68464668e-02, 5.36540848e-02, 6.14031251e-04])

In [26]:
df333 = churn_prob
churn_prob1 = pd.DataFrame(df333)
churn_prob1.to_csv('churn_prob2.csv')

In [29]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(train_x, train_y)
y_pred = xgb_model.predict(vali_x)
y_prob = xgb_model.predict_proba(vali_x)[:,1]

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:01:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [30]:
print("Accuracy:",metrics.accuracy_score(vali_y, y_pred))
print("AUC:",metrics.roc_auc_score(vali_y, y_prob))
print(metrics.classification_report(vali_y,y_pred))

Accuracy: 0.8293871866295265
AUC: 0.9001429165593999
              precision    recall  f1-score   support

           0       0.86      0.89      0.87     22005
           1       0.76      0.71      0.74     11023

    accuracy                           0.83     33028
   macro avg       0.81      0.80      0.81     33028
weighted avg       0.83      0.83      0.83     33028



In [19]:
from sklearn import linear_model
from sklearn import metrics 

clf = linear_model.LogisticRegression(penalty='l2')
clf.fit(train_x, train_y)

y_pred = clf.predict(vali_x)
y_prob = clf.predict_proba(vali_x)[:,1]

print("Accuracy:",metrics.accuracy_score(vali_y, y_pred))
print("AUC:",metrics.roc_auc_score(vali_y, y_prob))
print(metrics.classification_report(vali_y,y_pred))

Accuracy: 0.8294780186508417
AUC: 0.90352093454216
              precision    recall  f1-score   support

           0       0.85      0.89      0.87     21725
           1       0.78      0.71      0.74     11303

    accuracy                           0.83     33028
   macro avg       0.81      0.80      0.81     33028
weighted avg       0.83      0.83      0.83     33028



/Users/leon/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [57]:
revenue_x = train.drop(['churn'], axis=1)

In [59]:
log_coef_regular.to_csv('log_coef_regular.csv')

In [33]:
df111 = clf.coef_
log_coef = pd.DataFrame(df111)

In [67]:
df222 = y_prob
y_prob1 = pd.DataFrame(df222)
y_prob1.to_csv('y_prob1.csv')

In [36]:
log_coef.to_csv('log_coef.csv')

In [38]:
cuschurn_dummy.to_csv('cuschurn_dummy.csv')

In [52]:
import statsmodels.api as sm
logit_model=sm.Logit(train_x, train_y)

result=logit_model.fit()

print(result.summary2())

ValueError: endog must be in the unit interval.

In [50]:
LinearRegression(linear_model.LinearRegression)

/Users/leon/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


AttributeError: 'LogisticRegression' object has no attribute 'summary2'

In [ ]:
from sklearn import linear_model
from scipy import stats
import numpy as np


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self